In [32]:
import re
from typing import NamedTuple


class StateAction(NamedTuple):
    write: int
    move: int
    next_: str

    def __call__(self, tape):
        tape.value = self.write
        tape.move(self.move)
        return self.next_


class State(NamedTuple):
    action_0: StateAction
    action_1: StateAction
        
    def __call__(self, tape):
        return self[tape.value](tape)
    
    @classmethod
    def from_iterable(cls, it):
        actions = []
        for current in (0, 1):
            assert f'If the current value is {current}:' in next(it)
            value = int(re.search(r'- Write the value (\d)\.', next(it)).group(1))
            move = -1 if 'left' in next(it) else 1
            next_ = re.search(r'- Continue with state ([A-Z])\.', next(it)).group(1)
            actions.append(StateAction(value, move, next_))
        return cls(*actions)


def read_states(lines):
    # find start
    it = iter(lines)
    states = {}
    starting_state = None
    diagnostic_checksum = 0
    parser = re.compile(
        r'('
        r'Begin in state (?P<start>[A-Z])\.'
        r'|'
        r'Perform a diagnostic checksum after (?P<checksum>\d+) steps\.'
        r'|'
        r'In state (?P<state>[A-Z]):'
        r')').search
    for match in filter(None, map(parser, it)):
        parsed = match.groupdict()
        if parsed['start']:
            starting_state = parsed['start']
        elif parsed['checksum']:
            diagnostic_checksum = int(parsed['checksum'])
        else:
            name = parsed['state']
            states[name] = State.from_iterable(it)
    return starting_state, diagnostic_checksum, states


class Tape:
    def __init__(self):
        self._current = self._positive = [0]
        self._negative = []
        self._pos = 0

    @property
    def value(self):
        return self._current[abs(self._pos)]
    
    @value.setter
    def value(self, new_value):
        self._current[abs(self._pos)] = new_value
    
    def move(self, step):
        new_pos = self._pos + step
        if self._pos >= 0 and new_pos < 0:
            self._current = self._negative
        elif self._pos < 0 and new_pos >= 0:
            self._current = self._positive
        while len(self._current) <= abs(new_pos):
            # grow in batches, assume more will be needed on this end.
            self._current.extend([0] * 10)
        self._pos = new_pos
    
    def checksum(self):
        return sum(self._negative) + sum(self._positive)


def run_turing(states, start, checksum):
    tape = Tape()
    state = states[start]
    for _ in range(checksum):
        state = states[state(tape)]
    return tape.checksum()

In [33]:
test_start, test_checksum, test_states = read_states('''\
Begin in state A.
Perform a diagnostic checksum after 6 steps.

In state A:
  If the current value is 0:
    - Write the value 1.
    - Move one slot to the right.
    - Continue with state B.
  If the current value is 1:
    - Write the value 0.
    - Move one slot to the left.
    - Continue with state B.

In state B:
  If the current value is 0:
    - Write the value 1.
    - Move one slot to the left.
    - Continue with state A.
  If the current value is 1:
    - Write the value 1.
    - Move one slot to the right.
    - Continue with state A.
'''.splitlines())

assert run_turing(test_states, test_start, test_checksum) == 3

In [34]:
with open('inputs/day25.txt') as day25:
    start, checksum, states = read_states(day25)

In [35]:
print('Part 1:', run_turing(states, start, checksum))

Part 1: 4769
